# Optyx: A ZX-based Python library for networked quantum architectures

## Hong-Ou-Mandel effect

### Experiment definition

In [ ]:
from optyx.photonic import BBS

# diagram generators:
# beam-splitter
beam_splitter = BBS(0)
beam_splitter.draw()

In [ ]:
from optyx.photonic import Create

# diagram composition with function syntax
hong_ou_mandel = (
    Create(1) @ Create(1) >>
    beam_splitter
)

hong_ou_mandel.draw()

 ![HOM](./hom.png "Hong-Ou-Mandel Effect")

### Diagram evaluation

In [ ]:
# an amplitude (raw result of tensor contraction)
from optyx.classical import Select
(
    hong_ou_mandel >> Select(1, 1)
).eval().tensor.array

In [ ]:
hong_ou_mandel.eval().prob_dist()

## Qubit teleportation - function syntax and backends

In [ ]:
from optyx import qubit, bit
from optyx.qubits import Scalar

from optyx import Channel
from optyx.qubits import Z, X, H, Measure, Scalar, qubit, bit
from optyx.classical import BitControlledGate

### Define the protocol

In [ ]:
# function syntax
# CNOT from ZX generators

@Channel.from_callable(
  dom=qubit @ qubit, cod=qubit @ qubit
)
def cnot(a, b):
  c, d = Z(1, 2)(a)
  Scalar(2 ** 0.5)()
  return X(2, 1)(c, b), d

<img src="teleport.svg" alt="Teleportation Protocol" width="800" style="max-width:100%;">


In [ ]:
bell = Scalar(0.5 ** 0.5) @ Z(0, 2)

@Channel.from_callable(
  dom=qubit, cod=qubit
)
def teleportation(c):
  a, b = bell()
  aa, cc = cnot(a, c)
  c_ = Measure(1)(H()(cc))
  a_ = Measure(1)(aa)
  bb = BitControlledGate(X(1, 1, 0.5))(a_, b)
  return BitControlledGate(Z(1, 1, 0.5))(c_, bb)

In [ ]:
# function syntax avoids explicit swaps and identity wires

teleportation_monoidal_syntax = (
    qubit @ bell >>
    cnot @ qubit >>
    H() @ qubit ** 2 >>
    Measure(1) @ Measure(1) @ qubit >>
    bit @ BitControlledGate(X(1, 1, 0.5)) >>
    BitControlledGate(Z(1, 1, 0.5))
)

### Verify the protocol

In [ ]:
import numpy as np
from optyx.qubits import Id

# both implementations are equivalent
np.allclose(
    teleportation.eval().tensor.array,
    teleportation_monoidal_syntax.eval().tensor.array,
    Id(1).double().to_tensor().eval().array
)

In [ ]:
# backends (Perceval, Discopy, Quimb)

from optyx.core.backends import (
    DiscopyBackend,
    QuimbBackend
)

np.allclose(
    teleportation.eval(DiscopyBackend()).tensor.array,
    teleportation.eval(QuimbBackend()).tensor.array
)

## Fusion teleportation

Ursin, R., Jennewein, T., Aspelmeyer, M. et al. Quantum teleportation across the Danube. Nature 430, 849 (2004). https://doi.org/10.1038/430849a

<img src="./teleportation_danube.png" alt="Fusion teleportation across the Danube" width="1150px">

Graphically, the fusion measurement we would like to use, takes the following form:

<img src="./fusion_ii.png" alt="Fusion measurement" width="1200px">


where $\underline{a}, \underline{b}, \underline{c}, \underline{d}$ are the measurement outcomes as the measured photon numbers. 

$\underline{s} = \underline{a} \oplus \underline{b}$

$\underline{k} = \underline{s} (\underline{b} + \underline{d}) + \neg \underline s (1 - \frac{\underline{a} + \underline{b}}{2})$

### Define the protocol

In [ ]:
from optyx.photonic import DualRail

dual_rail_encoded_bell = (
    bell >>
    DualRail(1) @ DualRail(1)
)

In [ ]:
from optyx.classical import PostselectBit, BitControlledGate
from optyx.photonic import Phase
from optyx.photonic import HadamardBS, qmode

# postselect on fusion success
fusion_failure_processing = PostselectBit(1)

# apply the box if the control bit is 1, otherwise apply an identity channel
correction = BitControlledGate(
    HadamardBS() >>
    (Phase(0.5) @ qmode) >>
    HadamardBS()
)

In [ ]:
from optyx.photonic import FusionTypeII

@Channel.from_callable(
    dom=qubit, cod=qmode @ qmode
)
def fusion_teleportation(a):
    dual_rail_encoded_input = DualRail(1)(a)
    b, c, d, e = dual_rail_encoded_bell()
    s, k = FusionTypeII()(*dual_rail_encoded_input, b, c)
    fusion_failure_processing(s)
    dr_output_1, dr_output_2 = correction(k, d, e)
    return dr_output_1, dr_output_2

In [ ]:
from optyx.photonic import FusionTypeII

fusion_teleportation_monoidal_syntax = (
    DualRail(1) @ dual_rail_encoded_bell >>
    FusionTypeII() @ qmode**2 >>
    fusion_failure_processing @ correction
)

fusion_teleportation_monoidal_syntax.foliation().draw(figsize=(8, 8))


### Verify the protocol

In [ ]:
import numpy as np
from optyx.photonic import Id

array_teleportation = fusion_teleportation_monoidal_syntax.eval().tensor.array
array_dr = (DualRail(1) @ Scalar(0.5**0.5)).double().to_tensor().eval().array

np.allclose(array_teleportation[:, :, :2, :2, :2, :2], array_dr)


### Approximate contraction with Quimb and Cotengra

In [ ]:
def _flat(x):
    return np.asarray(x, dtype=complex).ravel()

def cosine_similarity(SU, SV):
    a, b = _flat(SU), _flat(SV)
    num = abs(np.vdot(a, b))
    den = np.linalg.norm(a) * np.linalg.norm(b)
    if den == 0:
        return 0.0
    return float(num / den)

In [ ]:
from cotengra import HyperCompressedOptimizer

# cosine similarity between the result of exact contaction and approximate contraction for different chis
errors = []
for chi in range(1, 6):
    optimiser = HyperCompressedOptimizer(
        chi=chi
    )
    error_for_chi = []
    for _ in range(10):
         error_for_chi.append(
             cosine_similarity(
                 fusion_teleportation.eval(QuimbBackend(optimiser)).tensor.array,
                 array_teleportation
             )
         )
    errors.append(np.median(error_for_chi))

import matplotlib.pyplot as plt
plt.plot(range(1, 6), errors, marker='o')
plt.grid()
plt.xlabel('Chi')
plt.ylabel('Average cosine similarity')
plt.title('Fusion-based Teleportation similarity vs Contraction Chi')


### Photon loss and channel fidelity

In [ ]:
from optyx.photonic import FusionTypeII, PhotonLoss

# photo loss is one of the main error sources in photonic quantum computing
def fusion_teleportation_with_photon_loss(p):
    @Channel.from_callable(
        dom=qubit, cod=qmode**2
    )
    def fusion_teleportation(a):
        dr_input_1, dr_input_2 = DualRail(1)(a)
        b, c, d, e = dual_rail_encoded_bell()
        # apply photon loss to all modes here:
        #-----------------------------------
        dr_input_1_loss, dr_input_2_loss, b_loss, c_loss, d_loss, e_loss = (
            PhotonLoss(p)(dr_input_1), PhotonLoss(p)(dr_input_2), PhotonLoss(p)(b), PhotonLoss(p)(c), PhotonLoss(p)(d), PhotonLoss(p)(e)
        )
        #-----------------------------------
        s, k = FusionTypeII()(dr_input_1_loss, dr_input_2_loss, b_loss, c_loss)
        fusion_failure_processing(s)
        output_rail_1, output_rail = correction(k, d_loss, e_loss)
        return output_rail_1, output_rail
    return fusion_teleportation

In [ ]:
from optyx.core.channel import Spider, Diagram

def get_perm(n):
    return sorted(sorted(list(range(n))), key=lambda i: i % 2)

def channel_fidelity(diagram_1, diagram_2):
    bell_1 = Channel.tensor(*[Spider(0, 2, ty) for ty in diagram_1.dom])
    permutation_1 = Diagram.permutation(get_perm(len(bell_1.cod)), bell_1.cod)

    bell_2 = Channel.tensor(*[Spider(0, 2, ty) for ty in diagram_2.dom])
    permutation_2 = Diagram.permutation(get_perm(len(bell_2.cod)), bell_2.cod)

    choi_1 = bell_1 >> permutation_1 >> (diagram_1 @ diagram_1.dom)
    choi_2 = bell_2 >> permutation_2 >> (diagram_2 @ diagram_2.dom)

    return (choi_1 >> choi_2.dagger()).eval().tensor.array


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

ps = np.linspace(0.0, 1.0, 20)

# compute fidelity for different photon loss probabilities
F_avg_vals = []
succ_probs = []
for p in ps:
    S_impl = fusion_teleportation_with_photon_loss(p)
    S_tgt  = fusion_teleportation_monoidal_syntax
    s = channel_fidelity(S_impl, S_tgt)

    succ_probs.append(s)

plt.figure()
plt.plot(ps, succ_probs, marker='o')
plt.grid(True)
plt.xlabel('Photon Survival Probability (p)')
plt.ylabel('Fidelity')
plt.title('Fusion-based Teleportation: Photon Survival Probability vs Fidelity')


## Distributed entanglement generation

![Distributed entanglement](./distributed_entanglement.png "An example of distributed entanglement generation")

Main, D., Drmota, P., Nadlinger, D.P. et al. Distributed quantum computing across an optical network link. Nature 638, 383–388 (2025). https://doi.org/10.1038/s41586-024-08404-x~



### Fusion and photon distinguishability

#### Define the protocol

In [ ]:
from optyx.qubits import Z, Scalar, Id, Measure
from optyx.photonic import DualRail
from optyx.classical import PostselectBit
from discopy.drawing import Equation

bell_state = Z(0, 2) @ Scalar(0.5 ** 0.5)

# generators introducing new qubits accepting internal states
internal_state_1 = [1, 0]
internal_state_2 = [0, 1]
dual_rail_encoding = lambda state: DualRail(1, internal_states=[state])
encoding_layer =  dual_rail_encoding(internal_state_1) @ dual_rail_encoding(internal_state_2)

# postselect on fusion success and no errors
post_select = PostselectBit(1) @ PostselectBit(0)

protocol = (
    bell_state @ bell_state >>
    Id(1) @ (encoding_layer >> FusionTypeII() >> post_select) @ Id(1)
)
measure = Measure(2)

Equation(protocol >> measure, bell_state >> measure).draw(figsize=(8, 8))

#### Define a set of internal states with varying degrees of distinguishability

In [ ]:
import math

# internal states - 2 dimensional - move further and further apart
def rotated_unit_vectors(n: int = 10):
    for i in range(n):
        theta = i * (math.pi / 2) / (n - 1)
        yield (math.cos(theta), math.sin(theta))

unit_vectors = list(rotated_unit_vectors(15))

#### Run the experiments

In [ ]:
from optyx.qubits import Discard

inner_product_states = []
inner_product_bell_states = []

result_bell = bell_state.eval().tensor.array.flatten()
result_bell = result_bell / np.linalg.norm(result_bell)

for vector in unit_vectors:
    encoding_layer =  dual_rail_encoding(internal_state_1) @ dual_rail_encoding(vector)
    experiment = bell_state @ bell_state >> Id(1) @ (encoding_layer >> FusionTypeII()
                                                                                >> post_select) @ Id(1)

    f = (experiment >> bell_state.dagger()).inflate(2).eval().tensor.array
    normalisation = (experiment >> Discard(2)).inflate(2).eval().tensor.array

    inner_product_states.append(np.inner(vector, internal_state_1))
    inner_product_bell_states.append(f/normalisation)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(inner_product_states, inner_product_bell_states, marker='o')
plt.xlabel('<Internal state 1 | Internal state 2>')
plt.ylabel('<Result | Bell State> (fidelity)')
plt.title('Fidelity of the resulting state with the perfect Bell state')
plt.grid(True)
plt.show()

## Interfacing with external libraries

### Graphix

Open graphs only for now (graph + measurements; desire to implement deterministically - no corrections)

In [ ]:
import graphix

circuit = graphix.Circuit(2)
circuit.cnot(0, 1)

pattern = circuit.transpile().pattern

pattern.draw_graph()

In [ ]:
simulator = graphix.simulator.PatternSimulator(pattern, backend="statevector")
graphix_result = simulator.run().psi.conj()

In [ ]:
from optyx import qubits

optyx_zx = qubits.Circuit(pattern)

optyx_res = (
    qubits.Ket("+")**2 >> optyx_zx
).eval().amplitudes()

In [ ]:
for keys in optyx_res.keys():
    assert np.isclose(optyx_res[keys], graphix_result[keys], atol=1e-6)

### Perceval circuits and processors

#### Define the protocol in Perceval

In [ ]:
import perceval as pcvl

p = pcvl.Processor("SLOS", 6)
p.add(0, pcvl.catalog["postprocessed cnot"].build_processor())

p.add(0, pcvl.BS.H())
p.add(0, pcvl.Detector.pnr())
p.add(1, pcvl.Detector.pnr())
p.add(2, pcvl.Detector.pnr())
p.add(3, pcvl.Detector.pnr())

ff_X = pcvl.FFCircuitProvider(
  2, 0, pcvl.Circuit(2)
)
ff_X.add_configuration(
  [0, 1], pcvl.PERM([1, 0])
)
p.add(2, ff_X)

phi = pcvl.P("phi")
ff_Z = pcvl.FFConfigurator(
  2, 3,
  pcvl.PS(phi),
  {"phi": 0}
).add_configuration(
  [0, 1],
  {"phi": np.pi}
)
p.add(0, ff_Z)

pcvl.pdisplay(p, recursive=True)

In [ ]:
from optyx.qubits import Ket

state = Ket("+") >> Z(1, 1, 0.3)
state_array = state.eval().tensor.array
state_array = state_array / np.linalg.norm(state_array)

#### Evaluate the protocol in Perceval

In [ ]:
to_transmit = (complex(state_array[0])*pcvl.BasicState([1, 0]) +
               complex(state_array[1])*pcvl.BasicState([0, 1]))

sg = pcvl.StateGenerator(pcvl.Encoding.DUAL_RAIL)
bell_state = sg.bell_state("phi+")

input_state = to_transmit * bell_state
p.min_detected_photons_filter(2)

input_state *= pcvl.BasicState([0, 0])

p.with_input(input_state)

In [ ]:
result_perceval = p.probs()

#### Convert to Optyx and simulate

In [ ]:
from optyx import Channel

optyx_diagram = Channel.from_perceval(p)

In [ ]:
from optyx.qubits import Scalar, Ket
from optyx.photonic import DualRail

bell_state = Z(0, 2) @ Scalar(0.5**0.5)
transmit = Ket("+") >> Z(1, 1, 0.3)

input_state = transmit @ bell_state

protocol = (
    input_state >>
    DualRail(3) >>
    Channel.from_perceval(p)
)

In [ ]:
result_optyx = protocol.eval().prob_dist()

In [ ]:
def check_dict_agreement(d1, d2, rtol=1e-5, atol=1e-8):
    for key in d1.keys() - d2.keys():
        assert np.isclose(d1[key], 0, rtol=rtol, atol=atol)
    for key in d2.keys() - d1.keys():
        assert np.isclose(d2[key], 0, rtol=rtol, atol=atol)
    for key in d1.keys() & d2.keys():
        assert np.isclose(d1[key], d2[key], rtol=rtol, atol=atol)

In [ ]:
check_dict_agreement(
    {tuple(k): v for k, v in dict(result_perceval["results"]).items()},
    result_optyx
)